In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.metrics import AUC
from keras.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot

In [4]:
# Data Loading
print("Loading data...")
X_model = np.load('../../data/X_timeseries_expanded.npy.zip')
Y_model = pd.read_csv('../../data/Y_model.csv').values

Loading data...


In [5]:
# Train Test Split
print("Splitting data...")
X_train, X_test, y_train, y_test = train_test_split(X_model['X_timeseries_expanded'], Y_model, test_size=0.2, random_state=42)

Splitting data...


In [6]:
# design network
# References: 
# - https://towardsdatascience.com/choosing-the-right-hyperparameters-for-a-simple-lstm-using-keras-f8e9ed76f046
# - https://stats.stackexchange.com/questions/242238/what-is-considered-a-normal-quantity-of-outliers
# - https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
hidden_nodes = int(2/3 * (X_train.shape[1] * X_train.shape[2]))
model = Sequential()
model.add(LSTM(hidden_nodes, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.compile(
    loss='mae',
    optimizer='adam',
    metrics=[AUC()]
)

2022-12-15 06:39:19.585477: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-15 06:39:19.586003: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [7]:
print("Training model...")
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test), 
    verbose=1,
    shuffle=False,
    callbacks=[
        EarlyStopping(
            monitor='val_auc',
            mode='max',
            patience=3,
        ),
        ModelCheckpoint(
            'lstm.h5',
            save_best_only=True,
            monitor='val_auc',
            mode='max'
        )
    ]
)

Training model...
Epoch 1/50


2022-12-15 06:39:31.359019: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-15 06:39:31.886081: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-15 06:39:32.121167: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-15 06:39:32.759161: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1427/20000 [=>............................] - ETA: 4:16:51 - loss: 0.0655 - auc: 0.5003

InvalidArgumentError: Graph execution error:

Detected at node 'assert_greater_equal/Assert/AssertGuard/Assert' defined at (most recent call last):
    File "/Users/hwanghyeongyu/miniforge3/envs/ai_learning/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/hwanghyeongyu/miniforge3/envs/ai_learning/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/hwanghyeongyu/miniforge3/envs/ai_learning/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/hwanghyeongyu/miniforge3/envs/ai_learning/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/hwanghyeongyu/miniforge3/envs/ai_learning/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/hwanghyeongyu/miniforge3/envs/ai_learning/l

In [ ]:
# plot history
pyplot.plot(history.history['auc'], label='train')
pyplot.plot(history.history['val_auc'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# Save model with pickle
import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)